In [1]:
%config IPCompleter.use_jedi = False
%pdb off
%load_ext viztracer
from viztracer import VizTracer
%load_ext autoreload
%autoreload 3
import sys
from pathlib import Path
import neptune # for logging progress and results
from pyphoplacecellanalysis.General.Batch.NeptuneAiHelpers import set_environment_variables, neptune_output_figures
neptune_kwargs = {'project':"commander.pho/PhoDibaLongShort2023",
'api_token':"eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxOGIxODU2My1lZTNhLTQ2ZWMtOTkzNS02ZTRmNzM5YmNjNjIifQ=="}

# required to enable non-blocking interaction:
%gui qt5

from copy import deepcopy
from numba import jit
import numpy as np
import pandas as pd
from attrs import define, field, fields, Factory

# Pho's Formatting Preferences
# from pyphocorehelpers.preferences_helpers import set_pho_preferences, set_pho_preferences_concise, set_pho_preferences_verbose
# set_pho_preferences_concise()

## Pho's Custom Libraries:
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphocorehelpers.Filesystem.open_in_system_file_manager import reveal_in_system_file_manager

# NeuroPy (Diba Lab Python Repo) Loading
# from neuropy import core
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.core.epoch import NamedTimerange, Epoch
from neuropy.core.ratemap import Ratemap
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder
from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass

## For computation parameters:
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.utils.dynamic_container import DynamicContainer
from neuropy.utils.result_context import IdentifyingContext
from neuropy.core.session.Formats.BaseDataSessionFormats import find_local_session_paths
from neuropy.core.neurons import NeuronType

from pyphocorehelpers.print_helpers import generate_html_string

# pyPhoPlaceCellAnalysis:
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import NeuropyPipeline # get_neuron_identities
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import export_pyqtgraph_plot
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_load_session, batch_extended_computations, SessionBatchProgress, batch_programmatic_figures, batch_extended_programmatic_figures
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import PipelineSavingScheme
from pyphoplacecellanalysis.Pho2D.matplotlib.visualize_heatmap import visualize_heatmap
import pyphoplacecellanalysis.External.pyqtgraph as pg

from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_perform_all_plots
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import JonathanFiringRateAnalysisResult
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import _find_any_context_neurons

# Plotting
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
_bak_rcParams = mpl.rcParams.copy()
# mpl.rcParams['toolbar'] = 'None' # disable toolbars
# %matplotlib qt
import pylustrator
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import plot_multiple_raster_plot, plot_raster_plot
from pyphoplacecellanalysis.General.Mixins.DataSeriesColorHelpers import DataSeriesColorHelpers
from pyphoplacecellanalysis.General.Mixins.DataSeriesColorHelpers import UnitColoringMode, DataSeriesColorHelpers
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import _build_default_tick, build_scatter_plot_kwargs
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Mixins.Render2DScrollWindowPlot import Render2DScrollWindowPlotMixin, ScatterItemData
from scripts.run_BatchAnalysis import post_compute_validate, _on_complete_success_execution_session
from PhoDiba2023Paper import PAPER_FIGURE_figure_1_add_replay_epoch_rasters, PAPER_FIGURE_figure_1_full, PAPER_FIGURE_figure_3


global_data_root_parent_path = find_first_extant_path([Path(r'W:\Data'), Path(r'/media/MAX/Data'), Path(r'/Volumes/MoverNew/data'), Path(r'/home/halechr/turbo/Data')])
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"

Automatic pdb calling has been turned OFF
build_module_logger(module_name="Spike3D.pipeline"):
	 Module logger com.PhoHale.Spike3D.pipeline has file logging enabled and will log to EXTERNAL\TESTING\Logging\debug_com.PhoHale.Spike3D.pipeline.log


# Load Pipeline

In [2]:
%pdb off
# ==================================================================================================================== #
# Load Data                                                                                                            #
# ==================================================================================================================== #

active_data_mode_name = 'kdiba'
local_session_root_parent_context = IdentifyingContext(format_name=active_data_mode_name) # , animal_name='', configuration_name='one', session_name=self.session_name
local_session_root_parent_path = global_data_root_parent_path.joinpath('KDIBA')

# [*] - indicates bad or session with a problem
# 0, 1, 2, 3, 4, 5, 6, 7, [8], [9], 10, 11, [12], 13, 14, [15], [16], 17, 
# curr_context: IdentifyingContext = good_contexts_list[1] # select the session from all of the good sessions here.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15') # DONE. Very good. Many good Pfs, many good replays.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43') # BAD
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31') # DONE, Good Pfs but no good replays
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6') # BAD
curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19') # DONE, GREAT, both good Pfs and replays!
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-13_15-22-3') # DONE, Good Pfs, no good epochs.
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_12-25-50') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-19_13-34-40') # BAD
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3') # BAD, Good Pfs strangely despite horrible map, no good epochs
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-12_15-25-59') # BAD, No Epochs
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-16_18-47-52')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-17_12-52-15')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-25_13-20-55')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-28_12-38-13')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_12-15-3')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_22-4-5')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54') # DONE

local_session_parent_path: Path = local_session_root_parent_path.joinpath(curr_context.animal, curr_context.exper_name) # 'gor01', 'one' - probably not needed anymore
basedir: Path = local_session_parent_path.joinpath(curr_context.session_name).resolve()
print(f'basedir: {str(basedir)}')

# Read if possible:
saving_mode = PipelineSavingScheme.SKIP_SAVING
force_reload = False

# Force write:
# saving_mode = PipelineSavingScheme.TEMP_THEN_OVERWRITE
# saving_mode = PipelineSavingScheme.OVERWRITE_IN_PLACE
# force_reload = True


## TODO: if loading is not possible, we need to change the `saving_mode` so that the new results are properly saved.


    # ==================================================================================================================== #
    # Load Pipeline                                                                                                        #
    # ==================================================================================================================== #
# with VizTracer(output_file=f"viztracer_{get_now_time_str()}-full_session_LOO_decoding_analysis.json", min_duration=200, tracer_entries=3000000, ignore_frozen=True) as tracer:
# epoch_name_includelist = ['maze']
epoch_name_includelist = None
active_computation_functions_name_includelist=[#'_perform_estimated_epochs_computation',  # AL:WAYS OFF
                                            '_perform_baseline_placefield_computation',
                                        # '_perform_time_dependent_placefield_computation', # AL:WAYS OFF
                                        '_perform_extended_statistics_computation',
                                        '_perform_position_decoding_computation', 
                                        '_perform_firing_rate_trends_computation',
                                        '_perform_pf_find_ratemap_peaks_computation',
                                        # '_perform_time_dependent_pf_sequential_surprise_computation' # AL:WAYS OFF
                                        '_perform_two_step_position_decoding_computation',
                                        # '_perform_recursive_latent_placefield_decoding' # AL:WAYS OFF
                                    ]
# active_computation_functions_name_includelist=['_perform_baseline_placefield_computation']

curr_active_pipeline = batch_load_session(global_data_root_parent_path, active_data_mode_name, basedir, epoch_name_includelist=epoch_name_includelist,
                                        computation_functions_name_includelist=active_computation_functions_name_includelist,
                                        saving_mode=saving_mode, force_reload=force_reload,
                                        skip_extended_batch_computations=True, debug_print=False, fail_on_exception=False) # , active_pickle_filename = 'loadedSessPickle_withParameters.pkl'


if not force_reload: # not just force_reload, needs to recompute whenever the computation fails.
    try:
        curr_active_pipeline.load_pickled_global_computation_results()
    except Exception as e:
        print(f'cannot load global results: {e}')

# 6m 1.1s
# 12m 15.6s


Automatic pdb calling has been turned OFF
basedir: W:\Data\KDIBA\gor01\two\2006-6-07_16-40-19
Loading loaded session pickle file results : W:\Data\KDIBA\gor01\two\2006-6-07_16-40-19\loadedSessPickle.pkl... done.
Loading pickled pipeline success: W:\Data\KDIBA\gor01\two\2006-6-07_16-40-19\loadedSessPickle.pkl.
property already present in pickled version. No need to save.
build_lap_only_short_long_bin_aligned_computation_configs(...):
	len(active_session_computation_configs): 1
	len(final_active_session_computation_configs): 1
using provided computation_functions_name_includelist: ['_perform_baseline_placefield_computation', '_perform_extended_statistics_computation', '_perform_position_decoding_computation', '_perform_firing_rate_trends_computation', '_perform_pf_find_ratemap_peaks_computation', '_perform_two_step_position_decoding_computation']
	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this i

In [3]:
## Post Compute Validate 2023-05-16:
from scripts.run_BatchAnalysis import post_compute_validate
post_compute_validate(curr_active_pipeline)

curr_active_pipeline.reload_default_computation_functions()
extended_computations_include_includelist=['long_short_fr_indicies_analyses', 'jonathan_firing_rate_analysis', 'long_short_decoding_analyses', 'long_short_post_decoding'] # do only specifiedl , 'long_short_rate_remapping'
force_recompute_global = force_reload
newly_computed_values = batch_extended_computations(curr_active_pipeline, include_includelist=extended_computations_include_includelist, include_global_functions=True, fail_on_exception=True, progress_print=True, force_recompute=force_recompute_global, debug_print=False)
if (len(newly_computed_values) > 0) and (saving_mode.value != 'skip_saving'):
	print(f'newly_computed_values: {newly_computed_values}. Saving global results...')
	try:
		# curr_active_pipeline.global_computation_results.persist_time = datetime.now()
		# Try to write out the global computation function results:
		curr_active_pipeline.save_global_computation_results()
	except Exception as e:
		print(f'\n\n!!WARNING!!: saving the global results threw the exception: {e}')
		print(f'\tthe global results are currently unsaved! proceed with caution and save as soon as you can!\n\n\n')
else:
	print(f'no changes in global results.')
# 10m 29.5s

were pipeline preprocessing parameters missing and updated?: False
included includelist is specified: ['long_short_fr_indicies_analyses', 'jonathan_firing_rate_analysis', 'long_short_decoding_analyses', 'long_short_post_decoding'], so only performing these extended computations.
Running batch_extended_computations(...) with global_epoch_name: "maze"
jonathan_firing_rate_analysis already computed.
long_short_fr_indicies_analyses already computed.
long_short_decoding_analyses already computed.
long_short_post_decoding already computed.
done with all batch_extended_computations(...).
no changes in global results.


In [4]:
# try to find the 'ripple_df.pkl' externally computed ripples file:
external_computed_ripple_filepath = curr_active_pipeline.sess.basepath.joinpath('ripple_df.pkl').resolve()
print(f'external_computed_ripple_filepath: {external_computed_ripple_filepath}, exists: {external_computed_ripple_filepath.exists()}')

## try the '.ripple.npy' ripples:
active_file_suffix = '.ripple.npy'
external_computed_ripple_filepath = curr_active_pipeline.sess.filePrefix.with_suffix(active_file_suffix).resolve()
print(f'external_computed_ripple_filepath: {external_computed_ripple_filepath}, exists: {external_computed_ripple_filepath.exists()}')


external_computed_ripple_filepath: W:\Data\KDIBA\gor01\two\2006-6-07_16-40-19\ripple_df.pkl, exists: True
external_computed_ripple_filepath: W:\Data\KDIBA\gor01\two\2006-6-07_16-40-19\2006-6-07_16-40-19.ripple.npy, exists: True


In [7]:
curr_active_pipeline.reload_default_display_functions()

neptuner = batch_perform_all_plots(curr_active_pipeline, enable_neptune=True)


https://app.neptune.ai/commander.pho/PhoDibaLongShortUpdated/
https://app.neptune.ai/commander.pho/PhoDibaLongShortUpdated/e/LS2023-185
shared_aclus: [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64]
long_only_aclus: [12, 21, 48]
short_only_aclus: [18, 19, 65]
shared_aclus: [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64]
long_only_aclus: [12, 21, 48]
short_only_aclus: [18, 19, 65]
include_includelist: ['maze1', 'maze2', 'maze']
long_epoch_name: maze1, short_epoch_name: maze2, global_epoch_name: maze
in `batch_perform_all_plots(...)`: batch_programmatic_figures(...) failed with exception: name 'plot_1D_placecell_validation' is not defined. C

# End Run

In [5]:
## long_short_decoding_analyses:
curr_long_short_decoding_analyses = curr_active_pipeline.global_computation_results.computed_data['long_short_leave_one_out_decoding_analysis']
## Extract variables from results object:
long_one_step_decoder_1D, short_one_step_decoder_1D, long_replays, short_replays, global_replays, long_shared_aclus_only_decoder, short_shared_aclus_only_decoder, shared_aclus, long_short_pf_neurons_diff, n_neurons, long_results_obj, short_results_obj, is_global = curr_long_short_decoding_analyses.long_decoder, curr_long_short_decoding_analyses.short_decoder, curr_long_short_decoding_analyses.long_replays, curr_long_short_decoding_analyses.short_replays, curr_long_short_decoding_analyses.global_replays, curr_long_short_decoding_analyses.long_shared_aclus_only_decoder, curr_long_short_decoding_analyses.short_shared_aclus_only_decoder, curr_long_short_decoding_analyses.shared_aclus, curr_long_short_decoding_analyses.long_short_pf_neurons_diff, curr_long_short_decoding_analyses.n_neurons, curr_long_short_decoding_analyses.long_results_obj, curr_long_short_decoding_analyses.short_results_obj, curr_long_short_decoding_analyses.is_global

# (long_one_step_decoder_1D, short_one_step_decoder_1D), (long_one_step_decoder_2D, short_one_step_decoder_2D) = compute_short_long_constrained_decoders(curr_active_pipeline, recalculate_anyway=True)
long_epoch_name, short_epoch_name, global_epoch_name = curr_active_pipeline.find_LongShortGlobal_epoch_names()
long_epoch_context, short_epoch_context, global_epoch_context = [curr_active_pipeline.filtered_contexts[a_name] for a_name in (long_epoch_name, short_epoch_name, global_epoch_name)]
long_session, short_session, global_session = [curr_active_pipeline.filtered_sessions[an_epoch_name] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_results, short_results, global_results = [curr_active_pipeline.computation_results[an_epoch_name]['computed_data'] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_computation_config, short_computation_config, global_computation_config = [curr_active_pipeline.computation_results[an_epoch_name]['computation_config'] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_pf1D, short_pf1D, global_pf1D = long_results.pf1D, short_results.pf1D, global_results.pf1D
long_pf2D, short_pf2D, global_pf2D = long_results.pf2D, short_results.pf2D, global_results.pf2D
decoding_time_bin_size = long_one_step_decoder_1D.time_bin_size # 1.0/30.0 # 0.03333333333333333

## Get global 'long_short_post_decoding' results:
curr_long_short_post_decoding = curr_active_pipeline.global_computation_results.computed_data['long_short_post_decoding']
expected_v_observed_result, curr_long_short_rr = curr_long_short_post_decoding.expected_v_observed_result, curr_long_short_post_decoding.rate_remapping
rate_remapping_df, high_remapping_cells_only = curr_long_short_rr.rr_df, curr_long_short_rr.high_only_rr_df
Flat_epoch_time_bins_mean, Flat_decoder_time_bin_centers, num_neurons, num_timebins_in_epoch, num_total_flat_timebins, is_short_track_epoch, is_long_track_epoch, short_short_diff, long_long_diff = expected_v_observed_result['Flat_epoch_time_bins_mean'], expected_v_observed_result['Flat_decoder_time_bin_centers'], expected_v_observed_result['num_neurons'], expected_v_observed_result['num_timebins_in_epoch'], expected_v_observed_result['num_total_flat_timebins'], expected_v_observed_result['is_short_track_epoch'], expected_v_observed_result['is_long_track_epoch'], expected_v_observed_result['short_short_diff'], expected_v_observed_result['long_long_diff']

jonathan_firing_rate_analysis_result = JonathanFiringRateAnalysisResult(**curr_active_pipeline.global_computation_results.computed_data.jonathan_firing_rate_analysis.to_dict())
(epochs_df_L, epochs_df_S), (filter_epoch_spikes_df_L, filter_epoch_spikes_df_S), (good_example_epoch_indicies_L, good_example_epoch_indicies_S), (short_exclusive, long_exclusive, BOTH_subset, EITHER_subset, XOR_subset, NEITHER_subset), new_all_aclus_sort_indicies, assigning_epochs_obj = PAPER_FIGURE_figure_1_add_replay_epoch_rasters(curr_active_pipeline)


good_example_epoch_indicies_L: [15 49]
good_example_epoch_indicies_S: [ 31  49  55  68  70  71  73  77  78  89  94 100 104 105 111 114 117 118
 122 123 131]


In [20]:
from pyphoplacecellanalysis.PhoPositionalData.plotting.placefield import plot_1D_placecell_validation
curr_active_pipeline.reload_default_display_functions()

In [21]:

active_identifying_session_ctx, active_session_figures_out_path, active_out_figures_list = batch_programmatic_figures(curr_active_pipeline)

shared_aclus: [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64]
long_only_aclus: [12, 21, 48]
short_only_aclus: [18, 19, 65]
shared_aclus: [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64]
long_only_aclus: [12, 21, 48]
short_only_aclus: [18, 19, 65]
include_includelist: ['maze1', 'maze2', 'maze']
long_epoch_name: maze1, short_epoch_name: maze2, global_epoch_name: maze
	 saved C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\EXTERNAL\Screenshots\ProgrammaticDisplayFunctionTesting\2023-07-06\kdiba_gor01_two_2006-6-07_16-40-19_BatchPhoJonathanReplayFRC_long_only_[12, 21, 48].png
include_includelist: ['maze1', 'maze2', 'maze']
long_epoch_name: maze1, sh

In [25]:
plt.switch_backend('Qt5Agg')
active_out_figures_list[0].show()

In [6]:
from PhoDiba2023Paper import main_complete_figure_generations
from pyphocorehelpers.general_helpers import disable_function_context
import matplotlib.pyplot as plt


# use_show_mode = True
use_show_mode = False

with disable_function_context(plt, "show"):
	if use_show_mode:
		plt.switch_backend('Qt5Agg')
		main_complete_figure_generations(curr_active_pipeline, defer_show=False, save_figure=True)
	else:
		plt.switch_backend('Agg')
		main_complete_figure_generations(curr_active_pipeline, defer_show=True, save_figure=True)


https://app.neptune.ai/commander.pho/PhoDibaLongShortUpdated/
https://app.neptune.ai/commander.pho/PhoDibaLongShortUpdated/e/LS2023-184
shared_aclus: [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64]
long_only_aclus: [12, 21, 48]
short_only_aclus: [18, 19, 65]
shared_aclus: [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64]
long_only_aclus: [12, 21, 48]
short_only_aclus: [18, 19, 65]
include_includelist: ['maze1', 'maze2', 'maze']
long_epoch_name: maze1, short_epoch_name: maze2, global_epoch_name: maze
in `batch_perform_all_plots(...)`: batch_programmatic_figures(...) failed with exception: name 'plot_1D_placecell_validation' is not defined. C

In [ ]:

# ## Still blocking at:
# File c:\Users\pho\repos\Spike3DWorkEnv\Spike3D\PhoDiba2023Paper.py:828, in PaperFigureTwo.fig_2_Theta_FR_matplotlib(Fig2_Laps_FR, defer_show, **kwargs)
#     823 # # Add grouping annotations
#     824 # ax.text((x[0] + x[1]) / 2, -0.25, '$L_x C$', ha='center', va='center')
#     825 # ax.text((x[2] + x[3]) / 2, -0.25, '$S_x C$', ha='center', va='center')
#     827 if not defer_show:
# --> 828     plt.show()
#     830 return MatplotlibRenderPlots(name="fig_2_Theta_FR_matplotlib", figures=[fig], axes=[ax], context=active_context)

# File c:\Users\pho\repos\Spike3DWorkEnv\Spike3D\PhoDiba2023Paper.py:822, in PaperFigureTwo.fig_2_Theta_FR_matplotlib(Fig2_Laps_FR, defer_show, **kwargs)
#     817 # # Add grouping annotations
#     818 # ax.text((x[0] + x[1]) / 2, -0.25, '$L_x C$', ha='center', va='center')
#     819 # ax.text((x[2] + x[3]) / 2, -0.25, '$S_x C$', ha='center', va='center')
#     821 if not defer_show:
# --> 822     plt.show()




In [ ]:
plt.close('all') # close all open figures when done to release memory

In [ ]:
plt.show()

In [ ]:
# curr_active_pipeline.active_config_names
curr_active_pipeline.active_configs

In [ ]:
curr_active_pipeline.active_config_names

# ['maze1_odd', 'maze2_odd',  'maze_odd',
#  'maze1_even','maze2_even', 'maze_even',
#  'maze1_any', 'maze2_any', 'maze_any']


In [ ]:
curr_active_pipeline.reload_default_display_functions()

In [ ]:
# ==================================================================================================================== #
# HELPERS: Interactive Components                                                                                      #
# ==================================================================================================================== #
from neuropy.utils.matplotlib_helpers import interactive_select_grid_bin_bounds_2D
# fig, ax, rect_selector, set_extents = interactive_select_grid_bin_bounds_2D(curr_active_pipeline, epoch_name='maze', should_block_for_input=True)
# required to enable non-blocking interaction:
# %gui qt5

# %lsmagic

In [ ]:
plt.isinteractive()


In [ ]:
plt.switch_backend('Qt5Agg')

In [ ]:

# with plt.ion():
grid_bin_bounds = interactive_select_grid_bin_bounds_2D(curr_active_pipeline, epoch_name='maze', should_block_for_input=True, should_apply_updates_to_pipeline=False)
print(f'grid_bin_bounds: {grid_bin_bounds}')
print(f"Add this to `specific_session_override_dict`:\n\n{curr_active_pipeline.get_session_context().get_initialization_code_string()}:dict(grid_bin_bounds=({(grid_bin_bounds[0], grid_bin_bounds[1]), (grid_bin_bounds[2], grid_bin_bounds[3])})),\n")



In [ ]:
## Common matplotlib figure needs:

# - [ ] Rendering to PyQt5-backed window (for display or potentially interaction)

# - [ ] Saving to file

# - [ ] Blocking for input (for interaction), only occasionally.

from neuropy.utils.result_context import overwriting_display_context

In [ ]:
# Display Helpers:

# [ ] Changing the Window Title

# [ ] Changing the Save File Path/Name

# [ ] Changing the various suptitle/titles/axes-titles displayed on the graph in various places



In [ ]:
import matplotlib.pyplot as plt

def extract_titles():
    fig = plt.gcf()
    titles = {}
    
    # Get the window title
    # fig.canvas.manager # .set_window_title(title_string) # sets the window's title
    

    # titles['window_title'] = fig.canvas.manager.window.title()

    
    # titles['window_title'] = plt.gcf().canvas.get_window_title()
    try:
        titles['window_title'] = fig.canvas.manager.window.windowTitle()
    except AttributeError as e:
        try:
            titles['window_title'] = fig.canvas.get_window_title() # try this one
        except Exception as e:
            try:
                titles['window_title'] = f"{fig.number or ''}"
            except Exception as e:
                raise e # unhandled exception
    except Exception as e:
        raise e
    
    # Get the suptitle
    suptitle = fig._suptitle.get_text() if fig._suptitle else None
    titles['suptitle'] = suptitle
    
    # Get the titles of each axis
    axes = fig.get_axes()
    for i, ax in enumerate(axes):
        title = ax.get_title()
        titles[f'axis_title_{i+1}'] = title
    
    return titles

# # Generate a sample plot
# plt.plot([1, 2, 3, 4], [1, 4, 9, 16])
# plt.title('Plot Title')
# plt.xlabel('X-axis')
# plt.ylabel('Y-axis')

# Call the function to extract titles
captured_titles = extract_titles()
print(captured_titles)




In [ ]:
# I need a context wrapper or something similar that can detect when a specific target function call is about to be performed, and pause execution before it happens.
## For example, I might want WatchHandler(plt.show) to pause execution before plt.show() is called anywhere in the subsequent code.

import inspect

# class WatchHandler:
# 	def __init__(self, target_func):
# 		_out_fig_2.target_func = target_func

# 	def __call__(self, *args, **kwargs):
# 		# Get the caller's frame
# 		frame = inspect.currentframe().f_back

# 		# Check if the target function is about to be called
# 		if frame.f_code.co_name == _out_fig_2.target_func.__name__ and frame.f_code.co_filename == self.target_func.__code__.co_filename:
# 			# Pause execution before the target function is called
# 			input("Execution paused. Press Enter to continue...")

# 		# Call the target function
# 		return self.target_func(*args, **kwargs)
	

import inspect

class WatchHandler:
	def __init__(self, target_func):
		self.target_func = target_func

	def __call__(self, *args, **kwargs):
		# Check if the target function is about to be called
		if "plt.show" in inspect.stack()[1][4][0]:
			# Pause execution before the target function is called
			input("Execution paused. Press Enter to continue...")

		# Call the target function
		return self.target_func(*args, **kwargs)

# Create a plot
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.plot([1, 2, 3], [4, 5, 6])

# Wrap plt.show() with WatchHandler
plt.show = WatchHandler(plt.show)

# Call plt.show()
plt.show()
WatchHandler(plt.show)


In [ ]:
import matplotlib.pyplot as plt


In [ ]:
plt.show()

# 2023-07-03 - Looking at directional placefields
The purpose of looking at this was to see if the bimodality actually should have belonged to directional placecells, and if splitting on direction would result in cleaner time spike rasters during replays instead of the strange bimodal setup I see now.
- [ ] I realized why my replay rasters don't look as clean and simple as yours despite the replays looking good. I think your unidirectional filter results in less bimodal placefields. 


In [ ]:
# ==================================================================================================================== #
# Show 1D Placefields for both Short and Long (top half of the figure)                                                 #
# ==================================================================================================================== #
active_identifying_session_ctx = curr_active_pipeline.sess.get_context() # 'bapun_RatN_Day4_2019-10-15_11-30-06'

long_single_cell_pfmap_processing_fn = None
short_single_cell_pfmap_processing_fn = None
# sort_idx = None
# sort_idx = sort_ind
# sort_idx = sort_indicies
# sort_idx = new_all_aclus_sort_indicies
sort_idx = 'peak_long'
# sort_idx = 'bad'

# include_includelist = ['maze1_odd', 'maze2_odd', 'maze_odd'] # Odd Only
# include_includelist = ['maze1_even','maze2_even', 'maze_even'] # Even laps only
# include_includelist = ['maze1_any','maze2_any', 'maze_any']

# active_identifying_session_ctx

# with plt.ion():
with plt.ioff():
	pf1d_compare_graphics = curr_active_pipeline.display('_display_short_long_pf1D_comparison', active_identifying_session_ctx.overwriting_context(laps='all'), include_includelist=['maze1_any','maze2_any', 'maze_any'], single_figure=False, debug_print=False, fignum='Short v Long pf1D Comparison - all laps',
									long_kwargs={'sortby': sort_idx, 'single_cell_pfmap_processing_fn': long_single_cell_pfmap_processing_fn},
									short_kwargs={'sortby': sort_idx, 'single_cell_pfmap_processing_fn': short_single_cell_pfmap_processing_fn, 'curve_hatch_style': {'hatch':'///', 'edgecolor':'k'}},
									save_figure=True,
									defer_render=False
									)

In [ ]:
with plt.ion():
	pf1d_compare_graphics = curr_active_pipeline.display('_display_short_long_pf1D_comparison', active_identifying_session_ctx.overwriting_context(laps='even'), include_includelist=['maze1_even','maze2_even', 'maze_even'], single_figure=False, debug_print=False, fignum='Short v Long pf1D Comparison - EVEN laps only',
									long_kwargs={'sortby': sort_idx, 'single_cell_pfmap_processing_fn': long_single_cell_pfmap_processing_fn},
									short_kwargs={'sortby': sort_idx, 'single_cell_pfmap_processing_fn': short_single_cell_pfmap_processing_fn, 'curve_hatch_style': {'hatch':'///', 'edgecolor':'k'}},
									save_figure=True,
									defer_render=False
									)

In [ ]:
with plt.ion():
	pf1d_compare_graphics = curr_active_pipeline.display('_display_short_long_pf1D_comparison', active_identifying_session_ctx.overwriting_context(laps='odd'), include_includelist=['maze1_odd', 'maze2_odd', 'maze_odd'], single_figure=False, debug_print=False, fignum='Short v Long pf1D Comparison - ODD laps only',
									long_kwargs={'sortby': sort_idx, 'single_cell_pfmap_processing_fn': long_single_cell_pfmap_processing_fn},
									short_kwargs={'sortby': sort_idx, 'single_cell_pfmap_processing_fn': short_single_cell_pfmap_processing_fn, 'curve_hatch_style': {'hatch':'///', 'edgecolor':'k'}},
									save_figure=True,
									defer_render=False
									)

In [ ]:
long_epoch_name, short_epoch_name, global_epoch_name = curr_active_pipeline.find_LongShortGlobal_epoch_names()

print(long_epoch_name, short_epoch_name, global_epoch_name)

# Common Display Setup

In [ ]:
import matplotlib
# configure backend here
matplotlib.use('Qt5Agg')
# backend_qt5agg
# %matplotlib qt5
# %matplotlib qt
# matplotlib.use('AGG') # non-interactive backend ## 2022-08-16 - Surprisingly this works to make the matplotlib figures render only to .png file, not appear on the screen!

import matplotlib as mpl
import matplotlib.pyplot as plt
_bak_rcParams = mpl.rcParams.copy()
# mpl.rcParams['toolbar'] = 'None' # disable toolbars

# Figure 1) pf1D Ratemaps, Active set, etc

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import plot_multiple_raster_plot, plot_raster_plot
from pyphoplacecellanalysis.Pho2D.matplotlib.visualize_heatmap import visualize_heatmap_pyqtgraph # used in `plot_kourosh_activity_style_figure`
from PhoDiba2023Paper import PAPER_FIGURE_figure_1_full, PAPER_FIGURE_figure_1_add_replay_epoch_rasters

pf1d_compare_graphics, (example_epoch_rasters_L, example_epoch_rasters_S), example_stacked_epoch_graphics = PAPER_FIGURE_figure_1_full(curr_active_pipeline) # did not display the pf1

In [ ]:
pf1d_compare_fig_L, pf1d_compare_fig_S = pf1d_compare_graphics.figures


In [ ]:
pf1d_compare_fig_L.show()

In [ ]:
plt.show()

In [ ]:
## Stacked Epoch Plot
example_stacked_epoch_graphics = curr_active_pipeline.display('_display_long_and_short_stacked_epoch_slices', defer_render=False, save_figure=True)

In [ ]:
from PhoDiba2023Paper import build_shared_sorted_neuronIDs

ratemap = long_pf1D.ratemap
included_unit_neuron_IDs = EITHER_subset.track_exclusive_aclus
rediculous_final_sorted_all_included_neuron_ID, rediculous_final_sorted_all_included_pfmap = build_shared_sorted_neuronIDs(ratemap, included_unit_neuron_IDs, sort_ind=new_all_aclus_sort_indicies.copy())

In [ ]:
heatmap_pf1D_win, heatmap_pf1D_img = visualize_heatmap_pyqtgraph(rediculous_final_sorted_all_included_pfmap, show_yticks=False, title=f"pf1D Sorted Visualization", defer_show=True)

In [ ]:
active_curves_sorted = long_pf1D.ratemap.normalized_tuning_curves[is_included][included_new_all_aclus_sort_indicies]
heatmap_pf1D_win, heatmap_pf1D_img = visualize_heatmap_pyqtgraph(active_curves_sorted, show_yticks=False, title=f"pf1D Sorted Visualization", defer_show=True)

In [ ]:
from pyphoplacecellanalysis.Pho2D.stacked_epoch_slices import DecodedEpochSlicesPaginatedFigureController

pagination_controller_L, pagination_controller_S = example_stacked_epoch_graphics.plot_data['controllers']
ax_L, ax_S = example_stacked_epoch_graphics.axes
final_figure_context_L, final_context_S = example_stacked_epoch_graphics.context

In [ ]:
from PhoDiba2023Paper import UserAnnotationsManager

user_anootations = UserAnnotationsManager.get_user_annotations()
user_anootations

In [ ]:
## Update the currently displayed selections with the user annotations:


## Capture current user selection
saved_selection_L = pagination_controller_L.save_selection()
saved_selection_S = pagination_controller_S.save_selection()

saved_selection_L = UserAnnotationsManager.update_selections_from_annotations(saved_selection_L, user_anootations)
saved_selection_S = UserAnnotationsManager.update_selections_from_annotations(saved_selection_S, user_anootations)
## re-apply the selections:
pagination_controller_L.restore_selections(saved_selection_L)
pagination_controller_S.restore_selections(saved_selection_S)


In [ ]:
# ==================================================================================================================== #
# Show Example Replay Epochs containing the long or short only cells                                                                  #
# ==================================================================================================================== #
(epochs_df_L, epochs_df_S), (filter_epoch_spikes_df_L, filter_epoch_spikes_df_S), (good_example_epoch_indicies_L, good_example_epoch_indicies_S), (short_exclusive, long_exclusive, BOTH_subset, EITHER_subset, XOR_subset, NEITHER_subset), new_all_aclus_sort_indicies, assigning_epochs_obj = PAPER_FIGURE_figure_1_add_replay_epoch_rasters(curr_active_pipeline)

# unit_colors_list = None # default rainbow of colors for the raster plots
neuron_qcolors_list = [pg.mkColor('black') for aclu in EITHER_subset.track_exclusive_aclus] # solid green for all
unit_colors_list = DataSeriesColorHelpers.qColorsList_to_NDarray(neuron_qcolors_list, is_255_array=True)

# Copy and modify the colors for the cells that are long/short exclusive:
unit_colors_list_L = deepcopy(unit_colors_list)
is_L_exclusive = np.isin(EITHER_subset.track_exclusive_aclus, long_exclusive.track_exclusive_aclus) # get long exclusive
unit_colors_list_L[0, is_L_exclusive] = 255 # [1.0, 0.0, 0.0, 1.0]
unit_colors_list_L[1, is_L_exclusive] = 0.0
unit_colors_list_L[2, is_L_exclusive] = 0.0

unit_colors_list_S = deepcopy(unit_colors_list)
is_S_exclusive = np.isin(EITHER_subset.track_exclusive_aclus, short_exclusive.track_exclusive_aclus) # get short exclusive
unit_colors_list_S[0, is_S_exclusive] = 0.0 # [1.0, 0.0, 0.0, 1.0]
unit_colors_list_S[1, is_S_exclusive] = 0.0
unit_colors_list_S[2, is_S_exclusive] = 255.0


## Build scatterplot args:    
# Common Tick Label
vtick = pg.QtGui.QPainterPath()

# Thicker Tick Label:
tick_width = 0.25
# tick_width = 10.0
half_tick_width = 0.5 * tick_width
vtick.moveTo(-half_tick_width, -0.5)
vtick.addRect(-half_tick_width, -0.5, tick_width, 1.0) # x, y, width, height
pen = {'color': 'white', 'width': 1}
override_scatter_plot_kwargs = dict(pxMode=True, symbol=vtick, size=6, pen=pen)


In [ ]:
defer_show = False
save_figure = True

In [ ]:
filter_epoch_spikes_df_L.spikes.rebuild_fragile_linear_neuron_IDXs();
filter_epoch_spikes_df_S.spikes.rebuild_fragile_linear_neuron_IDXs();

In [ ]:
## Build a real context:
active_identifying_session_ctx = curr_active_pipeline.sess.get_context() # 'bapun_RatN_Day4_2019-10-15_11-30-06'

#TODO 2023-07-05 16:50: - [ ] Add # @overwriting_display_context(fig='1', frs='Laps') to the Figure 1 function
active_identifying_session_ctx.adding_context(None, fig=1, track='long', epoch='example_replays')


In [ ]:
from neuropy.utils.result_context import IdentifyingContext

max_width=79

pprint.

In [ ]:
from neuropy.utils.result_context import providing_context, IdentifyingContext
from IPython.lib.pretty import pprint

# pprint

@providing_context(class_ctx='test_class_context', key1='a', key2='b')
@define(slots=False)
class TestClass:
#TODO 2023-07-05 18:45: - [ ] Class to test the context decorators ` providing_contex`, etc. Attempting to figure out how to get nesting to work
    """ DATENOW

    """
    val: int = field(default=0)
    
    @providing_context(func='test_func_A', key1='A')
    def test_func_A(self, *args, active_context=None, **kwargs):
        print(f"TestClass.test_func_A: kwargs: {kwargs}")
        # print(f'\tactive_context: {active_context.get_description(include_property_names=True)}')
        pprint(active_context)

        parent_context = type(self).active_context
        print(f'parent_context: ')
        pprint(parent_context)


    @providing_context(func='test_func_B', key1='B')
    def test_func_B(self, *args, active_context=None, **kwargs):
        @providing_context(subfn='_subfn_fn_B_1', key1='b', subkey1='_bb')
        def _subfn_fn_B_1():
            print(f"TestClass.test_func_B._subfn_fn_B_1: kwargs: {kwargs}")
            # print(f'\tactive_context: {active_context.get_description(include_property_names=True)}')
            pprint(active_context)

        print(f"TestClass.test_func_B: kwargs: {kwargs}")
        # print(f'\tactive_context: {active_context.get_description(include_property_names=True)}')
        pprint(active_context)
        _subfn_fn_B_1()


test = TestClass(val=3)
# print(test.active_context.get_description(include_property_names=True))
pprint(test.active_context)
# test.test_func_A(active_context=test.active_context)
# test.test_func_B(active_context=test.active_context)

test.test_func_A(active_context=None)
test.test_func_B(active_context=None)


In [ ]:
example_epoch_rasters_L = plot_multiple_raster_plot(epochs_df_L, filter_epoch_spikes_df_L, included_neuron_ids=EITHER_subset.track_exclusive_aclus, unit_sort_order=new_all_aclus_sort_indicies, unit_colors_list=unit_colors_list_L, scatter_plot_kwargs=override_scatter_plot_kwargs,
									epoch_id_key_name='replay_epoch_id', scatter_app_name="Long Decoded Example Replays", defer_show=defer_show)
app_L, win_L, plots_L, plots_data_L = example_epoch_rasters_L
plots_data_L.active_context = curr_active_pipeline.build_display_context_for_session('plot_multiple_raster_plot', fig=1, track='long', epoch='example_replays')
if save_figure:
	curr_active_pipeline.output_figure(plots_data_L.active_context, win_L) 

example_epoch_rasters_S = plot_multiple_raster_plot(epochs_df_S, filter_epoch_spikes_df_S, included_neuron_ids=EITHER_subset.track_exclusive_aclus, unit_sort_order=new_all_aclus_sort_indicies, unit_colors_list=unit_colors_list_S, scatter_plot_kwargs=override_scatter_plot_kwargs,
                                                                epoch_id_key_name='replay_epoch_id', scatter_app_name="Short Decoded Example Replays", defer_show=defer_show)
app_S, win_S, plots_S, plots_data_S = example_epoch_rasters_S
plots_data_S.active_context = curr_active_pipeline.build_display_context_for_session('plot_multiple_raster_plot', fig=1, track='short', epoch='example_replays')
if save_figure:
	curr_active_pipeline.output_figure(plots_data_S.active_context, win_S)

In [ ]:
curr_active_pipeline.output_figure(plots_data_S.active_context, win_S) # Error occured while writing .png for fig. 'GraphicsLayoutWidget' object has no attribute 'savefig'. Skipping.

In [ ]:
# Test single `plot_raster_plot` calls
an_epoch = list(epochs_df_L.itertuples())[0]
an_epoch_spikes_df = filter_epoch_spikes_df_L[filter_epoch_spikes_df_L['replay_epoch_id'] == an_epoch.Index]

_out_single_raster_plot = plot_raster_plot(an_epoch_spikes_df, included_neuron_ids=EITHER_subset.track_exclusive_aclus, unit_sort_order=None, unit_colors_list=unit_colors_list, scatter_plot_kwargs=override_scatter_plot_kwargs, scatter_app_name="test1")
_out_single_raster_plot2 = plot_raster_plot(an_epoch_spikes_df, included_neuron_ids=EITHER_subset.track_exclusive_aclus, unit_sort_order=new_all_aclus_sort_indicies, unit_colors_list=unit_colors_list, scatter_plot_kwargs=override_scatter_plot_kwargs, scatter_app_name="test2")

In [ ]:
app_alt, win_alt, plots_alt, plots_data_alt = plot_multiple_raster_plot(epochs_df_L, filter_epoch_spikes_df_L, included_neuron_ids=EITHER_subset.track_exclusive_aclus, unit_sort_order=None, unit_colors_list=unit_colors_list, scatter_plot_kwargs=override_scatter_plot_kwargs,
                                                                         epoch_id_key_name='replay_epoch_id', scatter_app_name="ALT Long Decoded Example Replays")

### Testing `plot_kourosh_activity_style_figure` for debugging:

In [ ]:
from pyphoplacecellanalysis.Analysis.Decoder.decoder_result import plot_kourosh_activity_style_figure
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.helpers import _helper_make_scatterplot_clickable

plot_aclus = EITHER_subset.track_exclusive_aclus.copy()
# plot_aclus = EITHER_subset.track_exclusive_aclus[new_all_aclus_sort_indicies].copy()
_out_A = plot_kourosh_activity_style_figure(long_results_obj, long_session, plot_aclus, unit_sort_order=new_all_aclus_sort_indicies, epoch_idx=13, callout_epoch_IDXs=None, skip_rendering_callouts=False)

In [ ]:
app, win, plots, plots_data = _out_A
# plots

In [ ]:
#TODO 2023-06-27 10:42: - [ ] Desperitely need a class that "explodes" the important variables and their types out of a DynamicParameters (dict-like) or other object.


In [ ]:
_out_n = plot_kourosh_activity_style_figure(long_results_obj, long_session, EITHER_subset.track_exclusive_aclus, unit_sort_order=new_all_aclus_sort_indicies, epoch_idx=49, callout_epoch_IDXs=np.arange(6), skip_rendering_callouts=False)

# Figure 2) Firing Rate Bar Graphs

In [26]:
# Instantaneous versions:
from PhoDiba2023Paper import PaperFigureTwo

_out_fig_2 = PaperFigureTwo(instantaneous_time_bin_size_seconds=0.01) # 10ms
_out_fig_2.compute(curr_active_pipeline=curr_active_pipeline, active_context=curr_active_pipeline.sess.get_context())
# _out_fig_2.display(defer_show=True, save_figure=True, active_context=curr_active_pipeline.sess.get_context())

good_example_epoch_indicies_L: [15 49]
good_example_epoch_indicies_S: [ 31  49  55  68  70  71  73  77  78  89  94 100 104 105 111 114 117 118
 122 123 131]


In [27]:
LxC_ReplayDeltaMinus, LxC_ReplayDeltaPlus, SxC_ReplayDeltaMinus, SxC_ReplayDeltaPlus = _out_fig_2.LxC_ReplayDeltaMinus, _out_fig_2.LxC_ReplayDeltaPlus, _out_fig_2.SxC_ReplayDeltaMinus, _out_fig_2.SxC_ReplayDeltaPlus
LxC_ThetaDeltaMinus, LxC_ThetaDeltaPlus, SxC_ThetaDeltaMinus, SxC_ThetaDeltaPlus = _out_fig_2.LxC_ThetaDeltaMinus, _out_fig_2.LxC_ThetaDeltaPlus, _out_fig_2.SxC_ThetaDeltaMinus, _out_fig_2.SxC_ThetaDeltaPlus

In [28]:
LxC_ThetaDeltaMinus # cell_agg_inst_fr_list

SpikeRateTrends(inst_fr_df_list=[               12   21   48
0    0.000000e+00  0.0  0.0
1    0.000000e+00  0.0  0.0
2    3.157968e-16  0.0  0.0
3    5.789607e-16  0.0  0.0
4    5.789607e-16  0.0  0.0
..            ...  ...  ...
525  2.110520e-15  0.0  0.0
526  1.584734e-15  0.0  0.0
527  7.224180e-16  0.0  0.0
528  2.758339e-15  0.0  0.0
529  7.182011e-16  0.0  0.0

[530 rows x 3 columns],       12            21            48
0    0.0  0.000000e+00  0.000000e+00
1    0.0  0.000000e+00  1.515825e-16
2    0.0  0.000000e+00  0.000000e+00
3    0.0  0.000000e+00  9.094947e-16
4    0.0  0.000000e+00  0.000000e+00
..   ...           ...           ...
721  0.0  5.494864e-16  0.000000e+00
722  0.0  1.863398e-15  6.063298e-16
723  0.0  1.275905e-15  9.094947e-16
724  0.0  1.716003e-15  6.063298e-16
725  0.0  1.549356e-16  0.000000e+00

[726 rows x 3 columns],       12            21   48
0    0.0  0.000000e+00  0.0
1    0.0  0.000000e+00  0.0
2    0.0  3.056294e-16  0.0
3    0.0  1.900988e-16  0

In [43]:
d = LxC_ThetaDeltaMinus.epoch_agg_inst_fr_list # (n_epochs, n_cells)
print(f'{d.shape}')

(40, 3)


In [46]:
%matplotlib qt
import seaborn as sns

sns.set_theme()

# https://seaborn.pydata.org/generated/seaborn.violinplot.html#seaborn.violinplot
# Show each distribution with both violins and points
sns.violinplot(data=d, palette="light:g", inner="stick", orient="h", split=True)
plt.show()

In [29]:
LxC_ThetaDeltaPlus

SpikeRateTrends(inst_fr_df_list=[      12            21   48
0    0.0  2.055635e-16  0.0
1    0.0  0.000000e+00  0.0
2    0.0  0.000000e+00  0.0
3    0.0  3.947460e-16  0.0
4    0.0  9.473903e-16  0.0
..   ...           ...  ...
662  0.0  0.000000e+00  0.0
663  0.0  4.428957e-16  0.0
664  0.0  0.000000e+00  0.0
665  0.0  4.486925e-16  0.0
666  0.0  0.000000e+00  0.0

[667 rows x 3 columns],       12            21            48
0    0.0  1.643130e-15  1.847411e-15
1    0.0  1.740830e-15  3.694822e-15
2    0.0  1.136868e-15  0.000000e+00
3    0.0  2.878776e-15  7.638334e-16
4    0.0  0.000000e+00  9.547918e-17
..   ...           ...           ...
778  0.0  1.039855e+02  1.761125e-15
779  0.0  8.828837e+01  7.889353e-16
780  0.0  3.499527e+01  1.834196e-15
781  0.0  6.285466e+00  4.115047e-15
782  0.0  4.699509e-01  0.000000e+00

[783 rows x 3 columns],       12            21   48
0    0.0  2.526374e-16  0.0
1    0.0  1.169915e-16  0.0
2    0.0  0.000000e+00  0.0
3    0.0  0.000000e+00  0

In [30]:
# Look at percent change
(LxC_ThetaDeltaPlus.cell_agg_inst_fr_list - LxC_ThetaDeltaMinus.cell_agg_inst_fr_list) / LxC_ThetaDeltaMinus.cell_agg_inst_fr_list

array([ 2.74079694, -0.1890868 , -0.55295022])

# Figure 3) `PAPER_FIGURE_figure_3`: Firing Rate Index and Long/Short Firing Rate Replays v. Laps

In [ ]:
_out, _out2 = PAPER_FIGURE_figure_3(curr_active_pipeline, defer_render=False, save_figure=True)

In [ ]:
_out2.figures
_out.axes

In [ ]:
fig = _out.figures
fig
# Computed long ($L$)|short($S$) firing rate indicies